In [4]:
%load_ext autoreload
%autoreload 2
from invest_data import load_data, stickers

data = load_data()

inception_date = {}
for s in stickers:
    d = data[s].head(1)
    inception_date[s] = d.index.to_pydatetime()
inception_date

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


{'D05.SI': array([datetime.datetime(2000, 1, 3, 0, 0)], dtype=object),
 'ME8U.SI': array([datetime.datetime(2010, 10, 21, 0, 0)], dtype=object),
 'AJBU.SI': array([datetime.datetime(2014, 12, 12, 0, 0)], dtype=object),
 'C2PU.SI': array([datetime.datetime(2007, 11, 7, 0, 0)], dtype=object),
 'BABA': array([datetime.datetime(2014, 9, 19, 0, 0)], dtype=object),
 'MSFT': array([datetime.datetime(1996, 12, 31, 0, 0)], dtype=object),
 'AAPL': array([datetime.datetime(1996, 12, 31, 0, 0)], dtype=object),
 'NVDA': array([datetime.datetime(1999, 1, 22, 0, 0)], dtype=object),
 'SPY': array([datetime.datetime(1996, 12, 31, 0, 0)], dtype=object),
 'ARKK': array([datetime.datetime(2014, 10, 31, 0, 0)], dtype=object),
 'ARKW': array([datetime.datetime(2014, 9, 30, 0, 0)], dtype=object),
 'QQQ': array([datetime.datetime(1999, 3, 10, 0, 0)], dtype=object),
 'ARKG': array([datetime.datetime(2019, 12, 27, 0, 0)], dtype=object),
 'ARKQ': array([datetime.datetime(2019, 12, 31, 0, 0)], dtype=object),
 'AR

In [5]:
import copy
from datetime import datetime, timedelta
import pandas as pd
from common import get_price

start_td = datetime.strptime('2015-1-1', '%Y-%m-%d')
end_td = datetime.now()
portfolio_queue = copy.deepcopy(stickers)
invest_records_df = pd.DataFrame()
for s in stickers:
    series = pd.Series({'unit_volum':0, 'unit_sum':0}, name=s)
    invest_records_df = invest_records_df.append(series)
    
td = start_td
while td < end_td:
    td += timedelta(days=30.4)
    if len(portfolio_queue) == 0:
        portfolio_queue = copy.deepcopy(stickers)
    invest_stock = portfolio_queue.pop()
    while td < inception_date[invest_stock]:
        invest_stock = portfolio_queue.pop()
    print(str(td) + '===> ' + invest_stock)
    
    price = get_price(data[invest_stock], td.strftime('%Y-%m-%d'))
    if price:
        invest_records_df['unit_volum'][invest_stock] += 1
        invest_records_df['unit_sum'][invest_stock] += price
    
invest_records_df

2015-01-31 09:36:00===> TCEHY
2015-03-02 19:12:00===> AMZN
2015-04-02 04:48:00===> GOOG
2015-05-02 14:24:00===> QQQ
2015-06-02 00:00:00===> ARKW
2015-07-02 09:36:00===> ARKK
2015-08-01 19:12:00===> SPY
2015-09-01 04:48:00===> NVDA
2015-10-01 14:24:00===> AAPL
2015-11-01 00:00:00===> MSFT
2015-12-01 09:36:00===> BABA
2015-12-31 19:12:00===> C2PU.SI
2016-01-31 04:48:00===> AJBU.SI
2016-03-01 14:24:00===> ME8U.SI
2016-04-01 00:00:00===> D05.SI
2016-05-01 09:36:00===> TCEHY
2016-05-31 19:12:00===> AMZN
2016-07-01 04:48:00===> GOOG
2016-07-31 14:24:00===> QQQ
2016-08-31 00:00:00===> ARKW
2016-09-30 09:36:00===> ARKK
2016-10-30 19:12:00===> SPY
2016-11-30 04:48:00===> NVDA
2016-12-30 14:24:00===> AAPL
2017-01-30 00:00:00===> MSFT
2017-03-01 09:36:00===> BABA
2017-03-31 19:12:00===> C2PU.SI
2017-05-01 04:48:00===> AJBU.SI
2017-05-31 14:24:00===> ME8U.SI
2017-07-01 00:00:00===> D05.SI
2017-07-31 09:36:00===> TCEHY
2017-08-30 19:12:00===> AMZN
2017-09-30 04:48:00===> GOOG
2017-10-30 14:24:00===

,unit_sum,unit_volum
D05.SI,77.014743,4.0
ME8U.SI,6.886546,4.0
AJBU.SI,5.035253,4.0
C2PU.SI,9.498217,4.0
BABA,561.079994,4.0
MSFT,335.622478,4.0
AAPL,142.822491,4.0
NVDA,477.666294,4.0
SPY,1271.988525,5.0
ARKK,217.159519,5.0


In [6]:
total_unit_sum = 0
total_current_sum = 0

for s in stickers:
    d = data[s].tail(1)
    current_price = d['Adj Close'][0]

    if invest_records_df['unit_volum'][s] ==0:
        continue
    percentage = current_price*invest_records_df['unit_volum'][s]/invest_records_df['unit_sum'][s]
    print(s + ' => ' + str(percentage))
    total_unit_sum += invest_records_df['unit_sum'][s]
    total_current_sum += current_price*invest_records_df['unit_volum'][s]
    
print('total' + '=> ' + str(total_current_sum/total_unit_sum))

D05.SI => 1.1192661788759697
ME8U.SI => 1.8528882869002699
AJBU.SI => 2.3355333624321992
C2PU.SI => 1.7266397714336519
BABA => 2.0358594872415345
MSFT => 2.4602642064684135
AAPL => 3.0930703394457786
NVDA => 4.361036445164274
SPY => 1.32088455230405
ARKK => 2.144967073219039
ARKW => 2.31949843213351
QQQ => 1.7526701827644646
ARKG => 1.2890195659562653
ARKQ => 1.2634573460508827
ARKF => 1.6703431435568157
GOOG => 1.8885556157746024
AMZN => 2.7056764848601014
TCEHY => 2.3662376080649175
total=> 2.271192987557852
